Finding the MRI brain tumor detection dataset

In [1]:
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np

print("Torch version:", torch.__version__)
print("NumPy version:", np.__version__)
print("OpenCV version:", cv2.__version__)


Torch version: 2.9.0+cpu
NumPy version: 2.1.2
OpenCV version: 4.12.0
